In [1]:
# This is an example of how we see 
# the package work. The functions listed here
# are probably the only ones that should be exposed, ie documented.
# others should br prepended with a double underscore
#  
# The cognet directory has the following "modules"
# which are seprate .py files containing clases and functions
# The modules are cognet.py, dataFormatter.py, model.py, util.py, viz.py
# we will write the viz.py later.
import sys

from quasinet.qnet import qdistance
from cognet.cognet import cognet as cg
from cognet.dataFormatter import dataFormatter
from cognet.model import model 
#import cognet.util
import pandas as pd
import numpy as np

yr = '2018'
POLEFILE='examples_data/polar_vectors.csv'
QPATH='examples_data/gss_'+yr+'.joblib'
IMMUTABLE_FILE='examples_data/immutable.csv'
GSSDATA = 'examples_data/gss_'+yr+'.csv'

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from cognet.util import assert_None, assert_array_dimension
class dataFormatter:
    """format data to be suitable for Qnet training and testing
    """

    def __init__(self,
                 samples):
        """init

        Args:
            samples ([str], optional): 2D array with rows as observations and columns as features.
        """
        self.samples = pd.read_csv(samples)
        self.features = {}
        self.nan_cols = []
        self.immutable_vars = None
        self.mutable_vars = None
        self.test_size = None
        self.random_state = None
        self.train_data = None
        self.test_data = None

    def __train_test_split(self,
                           test_size,
                           train_size=None,
                           random_state=None):
        """split the samples into training and testing samples

        Args:
          test_size (float): fraction of sample to take as test_size.
          train_size (float): fraction of sample to take as train_size. Defaults to None, and 1-test_size
          random_state (int, optional): random seed to split samples dataset . Defaults to None.
        """
        self.test_size = test_size
        self.random_state = random_state
        self.train_data, self.test_data = train_test_split(self.samples,
                                                           test_size=test_size,
                                                           train_size=train_size,
                                                           random_state=random_state)
    
    def Qnet_formatter(self,
                         samples=None,
                         key=None):
        """format data for Qnet input

        Args:
          samples ([str], optional): 2D array with rows as observations and columns as features.
          key (str): Either 'train' or 'test' key, to determine which set of features
        
        Returns:
            features and samples of either the train and test dataset
        """
        # if not isinstance(samples, np.ndarray):
        #     raise ValueError('Samples must be in numpy array form!')
        if samples is None:
            samples = self.samples
        features = np.array(samples.columns.astype(str)[:])
        samples = samples.values.astype(str)[:]
        # remove columns that are all NaNs
        not_all_nan_cols = ~np.all(samples == '', axis=0)
        self.nan_cols = np.all(samples == '', axis=0)

        samples = samples[:, not_all_nan_cols]
        
        features = features[not_all_nan_cols]
        features = list(features)
        if key is not None:
            self.features[key] = features
        return features, samples

    def format_samples(self,
                       key,
                       test_size=.5):
        """formats samples and featurenames, either all, train, or test
        
        Args:
          key (str): 'all', 'train', or 'test', corresponding to sample type

        Returns:
            samples and featurenames: formatted
        """
        
        
        if all(x is None for x in [self.train_data,
                                       self.test_data,
                                       self.samples]):
            raise ValueError("Split samples into test and train datasets or input samples first!")
        if key == 'train':
            self.__train_test_split(1-test_size)
            samples = self.train_data
        elif key == 'test':
            self.__train_test_split(test_size)
            samples = self.test_data
        elif key == 'all':
            samples = self.samples
        else:
            raise ValueError("Invalid key, key must be either 'all', 'test', or 'train")
        
        return self.Qnet_formatter(samples, key=key)
    
    def __set_varcase(self,
                      lower,
                      key='train',
                      vars=None):
        """set the features to all upper or lowercase

        Args:
          lower (bool): If true, set vars to lowercase, else to uppercase
          key (str, optional): Whether to set train or test features. Defaults to 'train'.
          vars ([str]): Mutable and immutable vars/features. Defaults to None.

        Returns:
          features, vars: formatted to either upper or lower case
        """
        if lower:
            features = [x.lower() for x in self.features[key]]
            if var is not None:
                vars = [x.lower() for x in vars]
        else:
            features = [x.upper() for x in self.features[key]]
            if vars is not None:
                vars = [x.upper() for x in vars]
        return features, vars

    def __interpretvars(self,
                        lower,
                        IMMUTABLE,
                        FILE=None,
                        LIST=None):
        """read in vars from file and set mutable, immutable

        Args:
          lower (bool): Whether to set variables to lowercase (True) or not (False)
          IMMUTABLE (book): IMMUTABLE if True, MUTABLE otherwise
          FILE (str, optional): file with vars in singular column. Defaults to None.
          LIST ([str], optional): 1D array of vars. Defaults to None.
          
        Returns:
          mutable vars, immutable vars: list
        """
        if IMMUTABLE:
            immutable_vars = np.array(LIST)
            if FILE is not None:
                immutable_vars = pd.read_csv(FILE,index_col=0).transpose()
            #assert_array_dimension(immutable_vars, 1)
            features, immutable_vars = self.__set_varcase(lower,
                                                          vars=immutable_vars)
            mutable_vars = [x for x in features
                            if x not in immutable_vars]
            immutable_vars = [x for x in immutable_vars
                              if x in features]
            invalid_vars = [x for x in immutable_vars
                            if x not in features]
        else:
            mutable_vars = LIST
            if FILE is not None:
                mutable_vars = pd.read_csv(FILE,index_col=0).transpose()
            #assert_array_dimension(mutable_vars, 1)
            features, mutable_vars = self.__set_varcase(lower,
                                                        vars=mutable_vars)
            immutable_vars = [x for x in features
                              if x not in mutable_vars]
            mutable_vars = [x for x in mutable_vars
                            if x in features]
            invalid_vars = [x for x in mutable_vars
                            if x not in features]
        if len(invalid_vars) != 0:
            print("{} vars not found".format(len(invalid_vars)))
            print("vars not found:{}".format(invalid_vars))
        return mutable_vars, immutable_vars

    def mutable_variables(self,
                immutable_list=None,
                IMMUTABLE_FILE=None,
                mutable_list=None,
                MUTABLE_FILE=None,
                lower=False):
        """set variables to be mutable or immutable

        Args:
          immutable_list (list)): 1D array of immutable variables. Defaults to None.
          IMMUTABLE_FILE (str, optional): file with immutable vars in singular column. Defaults to None.
          mutable_list (list, optional): 1D array of immutable variables. Defaults to None.
          MUTABLE_FILE (str, optional): file with mutable vars in singular column. Defaults to None.
          
        Returns:
          mutable_vars, immutable_vars: list
        """
        list_None = assert_None([immutable_list,mutable_list], raise_error=False)
        file_None = assert_None([IMMUTABLE_FILE,MUTABLE_FILE], raise_error=False)
        num_None = assert_None([immutable_list,mutable_list,
                                IMMUTABLE_FILE,MUTABLE_FILE], raise_error=False)
        if list_None == 0 or file_None == 0:
            raise ValueError("Only input either IMMUTABLE or MUTABLE vars, not both!")
        elif num_None == 4:
            raise ValueError("Too few inputs! One argument needed")
        elif num_None != 3:
            raise ValueError("Too many inputs! Only one argument needed")
        else:
            if IMMUTABLE_FILE is not None:
                mutable_vars, immutable_vars = self.__interpretvars(lower,
                                                                    IMMUTABLE=True,
                                                                    FILE=IMMUTABLE_FILE)
            elif MUTABLE_FILE is not None:
                mutable_vars, immutable_vars = self.__interpretvars(lower,
                                                                    IMMUTABLE=False,
                                                                    FILE=MUTABLE_FILE)
            elif immutable_list is not None:
                mutable_vars, immutable_vars = self.__interpretvars(lower,
                                                                    IMMUTABLE=True,
                                                                    LIST=immutable_list)
            elif mutable_list is not None:
                mutable_vars, immutable_vars = self.__interpretvars(lower,
                                                                    IMMUTABLE=False,
                                                                    LIST=mutable_list)
        self.mutable_vars, self.immutable_vars = mutable_vars, immutable_vars
        return mutable_vars, immutable_vars            

In [4]:
from quasinet.qnet import Qnet, load_qnet, save_qnet
from quasinet.qnet import export_qnet_tree, export_qnet_graph
from cognet.util import assert_None

class model:
    """Facilitate training and constructing Qnet
    """

    def __init__(self):
        """Init
        """
        self.myQnet = None
        self.features = None
        self.immutable_vars = None
        self.mutable_vars = None
        self.data_obj = None

    def fit(self,
            featurenames=None,
            samples=None,
            data_obj=None,
            min_samples_split=2,
            alpha=0.05,
            max_depth=-1,
            max_feats=-1,
            early_stopping=False,
            verbose=0,
            random_state=None,
            n_jobs=4):
        """fit Quasinet Qnet model

        Args:
          featurenames ([str], optional): names of the model features. Defaults to None.
          samples ([str], optional): 2D array with rows as observations and columns as features. Defaults to None.
          data_obj (obj, optional): Build Qnet directly from data obj without other inputs. Defaults to None.
          njobs (int, optional): Number of jobs used to fit Qnet. Defaults to 2.
        """
        num_None = assert_None([featurenames,samples,data_obj], raise_error=False)
        if num_None == 0:
            raise ValueError("input either samples and features or data object, not both!")
        elif data_obj is not None:
            featurenames, samples=data_obj.Qnet_formatter() # returns the training data
            print(len(samples))
            self.immutable_vars, self.mutable_vars = data_obj.immutable_vars, data_obj.mutable_vars
        elif num_None > 1:
            raise ValueError("input both samples and features or data object!")
        print("training Qnet -------------")
        self.myQnet = Qnet(n_jobs=njobs, feature_names=featurenames,
                           min_samples_split=min_samples_split, alpha=alpha,
                           max_depth=max_depth, max_feats=max_feats)
        self.myQnet.fit(samples)
        print("Qnet trained --------------")
        self.features = featurenames

    def save(self,
             file_path=None):
        """save qnet

        Args:
          file_path (str, optional): Desired Qnet filename. Defaults to None.
        """
        assert_None([self.myQnet])
        if file_path is None:
            file_path = 'tmp_Qnet.joblib'
        save_qnet(self.myQnet, file_path)
    
    def load(self,
             file_path,
             VERBOSE=False):
        """load Qnet from file

        Args:
          file_path (str): path to Qnet savefile
          VERBOSE (bool): boolean to turn on verbose

        Returns:
          [Qnet]: Qnet object
        """
        if VERBOSE:
            print("loading..")
        self.myQnet = load_qnet(file_path)
        self.features = self.myQnet.feature_names
        if VERBOSE:
            print("done")
        return self.myQnet

    def export_dot(self,
                   filename,
                   index=[3],
                   path='',
                   generate_trees=False,
                   threshold=0.2):
        """export Qnet trees

        Args:
          filename (str): Desired tree savefile
          index (list, optional): list of indices to generate trees. Defaults to [3].
          path (str, optional): Desired tree savefile path. Defaults to ''.
          generate_trees (bool, optional): Whether or not to generate individual trees. 
                                             Defaults to False, or to generate individual trees.
          threshold (float, optional): Numeric cutoff for edge weights. If the edge weights exceed 
                                         this cutoff, then we include it into the graph. Defaults to 0.2.
        """
        if not generate_trees:
            export_qnet_graph(self.myQnet, 
                              threshold, path+filename)
        else:
            for i in index:
                export_qnet_tree(self.myQnet,
                                 i, filename)


In [5]:
# ---------------------------------
# testing dataFormatter
data = dataFormatter(samples=GSSDATA)
# load the sample data
# have option for test/train split
# make checks to ensure we will not throw errors at qnet construction 
# data.train() returns training data
# data.test() returns test data
print(data.samples[:2])
features,samples = data.format_samples('train')

# we can set mutable and immutable vars from list or file
im_vars_df = pd.read_csv(IMMUTABLE_FILE, names=['vars'])
im_vars_list = im_vars_df.vars.to_list()
mutable_vars, immutable_vars = data.mutable_variables(immutable_list=im_vars_list)
mutable_vars, immutable_vars = data.mutable_variables(IMMUTABLE_FILE=IMMUTABLE_FILE)
# -------------------------------------

            wrkstat HRS1 HRS2 evwork        wrkslf  wrkgovt OCC10 PRESTG10  \
0  temp not working    e    c    NaN  someone else  private     b        c   
1  working fulltime    c    e    NaN  someone else  private     b        d   

  PRESTG105PLUS INDUS10  ...    neisafe rlooks rgroomed rweight rhlthend wtss  \
0             c       c  ...  very safe    NaN      NaN     NaN      NaN    e   
1             d       c  ...  very safe    NaN      NaN     NaN      NaN    c   

  wtssnr wtssall vstrat vpsu  
0      e       e   3301    1  
1      c       c   3301    1  

[2 rows x 1034 columns]


In [ ]:
# -------------------------------------
# testing model functionality
# can either input features and samples directly, or infer from data obj
model_ = model()

# qnet construction parameters
test_model_buildqnet = True
# infer qnet
if test_model_buildqnet:
        print("fitting")
        model_.fit(data_obj=data, njobs=6)
        print("fitted")
        model_.export_dot("tmp_dot_modelclass.dot",
                        generate_trees=True)
        model_.save("tmp_nodelclass.joblib")
        #model_.load("tmp_nodelclass.joblib")
else:
    model_.load("examples_data/gss_2018.joblib")
# -------------------------------------

fitting
1784
training Qnet -------------


In [ ]:
# -------------------------------------
# testing cognet
# set some paramaters in instantiating cognet class 
# if loading from model obj, no need to load_data, otherwise, load_data

Cg = cg()
print(len(model_.features))
Cg.load_from_model(model_, data, 'all')

# distance calculation for individual samples    
# we have a nsteps parameter (for sample 1 and sample2)
# which qsamples the sample1 and sample2 if set before
# computing distance. Note qsampling must only 
# change mutable varaibles, so need to compute base-freq
distance = Cg.distance(samples[1],samples[3],nsteps1=5, nsteps2=5)
print(distance)
qdistance_ = qdistance(samples[1],samples[3],Cg.qnet,Cg.qnet)
print("actual:{}".format(qdistance_))
# -------------------------------------

In [ ]:
#------------------
# produce stats on how many column names actually match
stats = Cg.set_poles(POLEFILE,"R","L",steps=120)

# compute polar distance matrix
dmatrix = Cg.polar_separation(nsteps=0)
#------------------

In [ ]:
#------------------
# the following are for single samples
dissonance_array = Cg.dissonance(1)
returndict = {}
rederr,r_prob,rand_err,s,qs,s_rand,mask_ = Cg.randomMaskReconstruction(1, returndict)# sample=np.array(samples[1]))

#ideology_index = Cg.compute_DLI_sample(3)
Cg.num_qsamples = 5
ideology_index = Cg.ideology(3,pole_1="R",pole_2="L")

# get dispersion of an individual sample
Dispersion_ = Cg.dispersion(3)
print(Dispersion_)
# compute distance from each pole
array_distances = Cg.polarDistance(1, returndict)
#-------------------

In [ ]:
#-------------------
# the following are for arrays of samples
# multiprocessing suffices
count = 0
if count == 0:
    Cg.set_nsamples(10)
    count +=1
# computing polar_indices makes sure that dissonance matrix only takes in polar cols
Cg.compute_polar_indices()
dissonance_array = Cg.dissonance_matrix(output_file='examples_results/DISSONANCE_matrix.csv')
# multiprocessing suffices
dataframes,error_array = Cg.randomMaskReconstruction_multiple('examples_results/randomMaskRecon_test.csv')
# multiprocessing suffices
ideology_index = Cg.compute_DLI_samples('ideology','examples_results/ideology.csv')
# multiprocessing suffices
local_dispersion = Cg.compute_DLI_samples('dispersion', 'examples_results/dispersion_test.csv')
# compute distance from each pole
# multiprocessing suffices
array_distances = Cg.polarDistance_multiple('examples_results/polarDistance_multiple_test.csv')
#-------------------

In [ ]:
# compute qdistance matrix for small set of samples
# set nsamples first to set the number of samples to be included in matrix
distance_matrix=Cg.distfunc_multiples("examples_results/distfunc_multiples_testing.csv")


In [ ]:
# write files to compute qdistance matrix for large set of samples
# execute generated shell script to run mpi parallelization on midway
Cg.dmat_filewriter("GSS_cognet.py", "examples_data/gss_2018.joblib",
                   MPI_SETUP_FILE="GSS_mpi_setup.sh",
                   MPI_RUN_FILE="GSS_mpi_run.sh",
                   MPI_LAUNCHER_FILE="GSS_mpi_launcher.sh",
                   YEARS='2018',NODES=4,T=14)

In [ ]:
## embedding
## embed generated Qdist Matrix
Cg.year = '2018'
Cg.embed('examples_results/distfunc_multiples_testing.csv', 'embed', 'examples_results/',EMBED_BINARY='cognet/cognet/bin/__embed__.so')
#pd.read_csv('examples_results/embed_E_2018.csv')
